В заданиях ниже используется файл "Opros_Osnovy_Python_recoded2.csv" так же прикреплённый в мудле. При тестировании гипотез уровень значимости принимаем равным 0.05.

_Задание 1_. Т-тест (критерий Стьюдента) используется для проверки гипотезы о равенстве средних случайных выборок из нормального закона распределения (изначально - с одинаковой дисперсией, но затем была приумана модификация Уэлча для различных дисперсий). Этот тест был разработан химиком пивоваренной компании Гинесс Уильямом Госсетом, занимавшимся, в частности, сравнением урожайности новых сортов ячменя. Специфика задачи подразумевает очень небольшие по размеру выборки, однако, опираясь на априорные требования нормальности, Госсету удалось создать довольно чувствительный тест. Так как политика компании Гинесс запрещала своим учёным публиковать полученные результаты, Госсет публиковался под псевдонимом Стьюдент, под этим же псевдонимом обрёл бессмертие.

Т-тест существует в трёх вариантах:

1 - одновыборочный, в котором на выборке X тестируется гипотеза H0 о том, что истинное матожидание равно некоторой константе a. Функция scipy.stats.ttest_1samp(X, a).

2 - двухвыборочный для двух независимых наблюдений X и Y, в котором тестируется гипотеза H0 о том, что истинные матожидания в популяциях, откуда взяты обе выборки, равны. Выборки могут быть разного размера. Функция scipy.stats.ttest_ind(X, Y). NB: если предполагается, что истинные дисперсии для двух выборок равны (гипотеза тестируется F-тестом (критерий Фишера)), то это можно указать при вызове функции в параметре equal_var. В этом случае критерий чувствительнее.

3 - двухвыборочный тест для двух зависимых наблюдений X1 и X2, в котором тестируется гипотеза H0 о том, что истинное матожидание разницы X1-X2 равно 0 (то есть отличия между двумя переменными в среднем нет). Зависимые наблюдения это наблюдения за объектами из одной и той же выборки (например, измеряется температура больных до и после приёма лекарства или для учеников сравнивается их средний бал по Русскому языку и средний бал по Литературе). Выборки, естественно, должны быть одинакового размера. Функция scipy.stats.ttest_rel(X1, X2).

Однако, прежде, чем использовать Т-тест, необходимо проверить гипотезу о нормальности. Среди столбцов измерений любви к различным видам еды (в нашем датафрейме) найдите два таких, для которых выполняется критерий нормальности. Функция scipy.stats.normaltest(X), нулевая гипотеза - выборка получена из нормального распределения.

In [2]:
import pandas as pd
import scipy.stats as st

df = pd.read_csv("Opros _Osnovy_Python_recoded2.csv", index_col = 0)

potato_list = ['жареный_картошка', 'вареный_картошка', 'печеный_картошка', 'тушеный_картошка', 'жареный_курица', 'вареный_курица','печеный_курица', 'тушеный_курица', 'жареный_кабачок', 'вареный_кабачок', 'печеный_кабачок', 'тушеный_кабачок', 'жареный_баклажан', 'вареный_баклажан', 'печеный_баклажан', 'тушеный_баклажан']
potato = df[potato_list]

for o in potato_list:
    x = st.normaltest(df[o]).pvalue
    if x > 0.05:
        print('Выборка получена из нормального распределения: ', o)

Выборка получена из нормального распределения:  жареный_картошка
Выборка получена из нормального распределения:  вареный_картошка
Выборка получена из нормального распределения:  тушеный_картошка
Выборка получена из нормального распределения:  печеный_кабачок
Выборка получена из нормального распределения:  тушеный_кабачок
Выборка получена из нормального распределения:  вареный_баклажан
Выборка получена из нормального распределения:  тушеный_баклажан


Отлично. Так как обе переменные у нас измерены для одних и тех же объектов, то нужно использовать критерий Стьюдента для зависимых выборок. Протестируйте равенство стредних и интерпретируйте результат. Постарайтесь найти две такие переменные, для которых гипотеза о равенстве матожиданий не отвергается.

In [43]:
potato_list_normal = [ 'жареный_картошка', 'вареный_картошка', 'тушеный_картошка', 'печеный_кабачок', 
               'тушеный_кабачок', 'вареный_баклажан', 'тушеный_баклажан' ]


n = len(potato_list_normal)
count_ttest = 0
for i in range(n):
    for j in range(i+1, n):
        X1 = df[potato_list_normal[i]]
        X2 = df[potato_list_normal[j]]
        if st.stats.ttest_rel(X1, X2).pvalue > 0.05: # применяем нулевую гипотезу
            print(potato_list_normal[i], ' ', potato_list_normal[j])
            count_ttest +=1
            
print( st.stats.ttest_rel(df['вареный_картошка'], df['тушеный_картошка']) )
# pvalue=0.46554885275237223, значит, гипотеза H0 о том, что истинное матожидание разницы X1-X2 равно 0 принята.

вареный_картошка   тушеный_картошка
тушеный_картошка   тушеный_баклажан
печеный_кабачок   тушеный_кабачок
печеный_кабачок   тушеный_баклажан
тушеный_кабачок   тушеный_баклажан
Ttest_relResult(statistic=0.742652027147428, pvalue=0.46554885275237223)


_Задание 2_. Если выборки распределены не нормально, или же переменные являются ранговыми, то для сравнения средних можно использовать непараметрические аналоги критерия Стьюдента: критерий Манна-Уитни (для независимых выборок, функция scipy.stats.mannwhitneyu(X, Y)) и критерий Вилкоксона (для зависимых выборок, функция scipy.stats.wilcoxon(X1, X2)). Эти критерии тестирует гипотезу не о равенстве матожиданий, а о равенстве медиан.

Итак, найдите два переменные (из тех, что о любви к еде), отличащиеся от нормально распределённых. Примените критерий Вилкоксона для сравнения их медиан и интерпретируйте результат.

In [41]:
potato_list_not_normal = ['жареный_курица', 'вареный_курица','печеный_курица',
               'тушеный_курица', 'жареный_кабачок', 'вареный_кабачок', 'жареный_баклажан', 'печеный_баклажан']

X1 = df['жареный_курица']
X2 = df['вареный_кабачок']
st.wilcoxon(X1, X2)
# Гипотеза H0 о равенстве медиан должна быть отклонена для жареный_курица и вареный_кабачок

WilcoxonResult(statistic=6.5, pvalue=0.0014084151641154605)

Теперь примените критерий Вилкоксона к тем же переменным, к которым вы применяли критерий Стьюдента и приняли гипотезу о равенстве матожиданий. Отличаются ли p-value? Как вы думаете, почему?

In [42]:
# вареный_картошка   тушеный_картошка
X1 = df['вареный_картошка']
X2 = df['тушеный_картошка']
st.wilcoxon(X1, X2)
# Гипотеза о равенстве медиан принята. В данном случае мы рассматриваем распределения близкие к нормальным, 
# а у нормального распределения медиана равна матожиданию (равенство матожиданий проверяли в критерии Стьюдента)

WilcoxonResult(statistic=42.5, pvalue=0.5229032345126707)

_Задание 3*_. Для всех переменных (о любви к еде) проверить гипотезу о нормальности. Для всех пар переменных, для которых гипотеза о нормальности выполнена, провести Т-тест. Ко всем парам перемнных применить критерий Вилкоксона. Оценить, для какой доли сравнений критерий Стьюдента и критерий Вилкоксона дают разный результат. Все результаты можно выводить в виде matplotlib.pyplot.matshow


In [50]:
n = len(potato_list)
count_wilc = 0
for i in range(n):
    for j in range(i+1, n):
        X1 = df[potato_list[i]]
        X2 = df[potato_list[j]]
        if st.wilcoxon(X1, X2).pvalue > 0.05:
            print(potato_list[i], ' ', potato_list[j])
            count_wilc +=1
print ('Доля тест Стьюдента ', count_ttest/n )
print ( 'Доля Вилкоксон ', count_wilc/n )

вареный_картошка   тушеный_картошка
тушеный_картошка   тушеный_баклажан
печеный_кабачок   тушеный_кабачок
печеный_кабачок   тушеный_баклажан
тушеный_кабачок   тушеный_баклажан
Доля тест Стьюдента  0.7142857142857143
Доля Вилкоксон  0.7142857142857143


_Задание 4_. Для проверки наличия связи между двумя номинальными переменными X1 и X2 используются точный тест Фишера и критерий Хи-кадрат. В Python точный тест Фишера реализован только для бинарных номинальных переменных (то есть каждая переменная принимает одно из двух значений, например "Мужчина/женщина" или "Болен/Здоров"). Критерий Хи-квадрат корректно применять только если для каждого возможного сочетания значений переменных X1 и X2 имеется не менее 5 наблюдений (но мы, в качестве тренировки, применим его и так). Пример использования обоих критериев приведён в файле "Python Day 3.ipynb", прикреплённом в мудле (в конце файла). Для обоих критериев нулевой гипотезой является независимость переменных X1 и X2.

Совместное действие двух количественных переменных X1 и X2 записывается в виде матрицы сопряжённости T12. Для получения матрицы сопряжённости двух переменных используется функция pandas.crosstab(X1, X2) (см. файл с примером).

Для проведения точного теста фишера используется функция scipy.stats.fisher_exact(T12). Функция возвращает кортеж (oddsratio, pvalue), где pvalue это p-value для гипотезы H0, а oddsratio это отношение шансов (ОШ), являющееся количественной мерой взаимосвязи. Чем сильнее ОШ отличается от 1 (в большую или меньшую сторону) там сильнее зависимость переменных. Подробнее про него можно почитать здесь: https://ru.wikipedia.org/wiki/%D0%9E%D1%82%D0%BD%D0%BE%D1%88%D0%B5%D0%BD%D0%B8%D0%B5_%D1%88%D0%B0%D0%BD%D1%81%D0%BE%D0%B2

Для применения критерия Хи-квадрат используется функция scipy.stats.chi2_contingency(T12), возвращающая кортеж (chi2, p, dof, expected), где chi2 - значение статистики хи-квадрат, p - p-value для нулевой гипотезы, dof - число степеней свободы (параметр хи-квадрат распределения) и expected - ожидаемая идеальная таблица сопряжённости при условии выполнения нулевой гипотезы. Нас здесь интересует только p и expected. Ожидаемая таблица сопряжённости используется для того, чтобы оценить величину отличия наблюдаемых данных от гипотетических (в смысле гипотезы H0).

Используя точный тест Фишера, проверьте гипотезу о том, что то, является человек гуманитарием или ITшником не зависит от пола. Проинтерпретируйте полученный результат. Так же проверьте, есть ли взаимосвязь между направлением подготовки и тем, поделился ли человек своим id ВКонтакте или нет (переменная "есть\_vk\_id").

In [5]:
T12_1 = pd.crosstab(df['пол'], df['направление']) # Таблица сопряженности.
x1 = st.fisher_exact(T12_1)
# ОШ - oddsratio отличается от 1, 22.5 > 1, т.е. сильная зависимость.
# p = 0.003 < 0.05 нулевая гипотеза (независимость признаков) отклонена.
print(x1)

T12_2 = pd.crosstab(df['есть_vk_id'], df['направление'])
x2 = st.fisher_exact(T12_2)
print(x2)
# ОШ - oddsratio отличается от 1, 0.4 < 1, но близко, нет зависимости (?). 
# Если посмотреть на p = 0.413 > 0.05, поэтому нулевая гипотеза (независимость признаков) принята.

(22.5, 0.0033289499570291017)
(0.40816326530612246, 0.4136492125454295)


Используя критерий Хи-квадрат сравните любую пару из переменных блока "зн_..." (Ответы на вопрос "Знаете ли вы") с любой другой номинальной переменной. Проинтерпретируйте результат. 

In [86]:
T12_3 = pd.crosstab(df['знак_пандас'], df['направление'])
x3 = st.chi2_contingency(T12_3)
print(x3)
# p = 0.15231292126322496 > 0.05, поэтому нулевая гипотеза (независимость признаков) принята.

(3.7636363636363637, 0.15231292126322496, 2, array([[5.73913043, 6.26086957],
       [2.39130435, 2.60869565],
       [2.86956522, 3.13043478]]))


_Задание 5*_. С помощью критерия хи-квадрат проверьте попарную незвисимость всех номинальных переменных. Проинтерпретируйте результат.

In [24]:
catvars = ['зн_переменная', 'зн_идентификатор', 'зн_функция',
       'зн_класс', 'зн_объект', 'зн_оператор', 'зн_операция', 'зн_рекурсия',
       'зн_массив', 'зн_выражение', 'зн_пайтон', 'зн_си', 'знак_ар',
       'знак_пайтон', 'знак_си', 'знак_пандас', 'знак_сайпай', 'знак_реквестс',
       'знак_паймонго', 'знак_теворкс', 'знак_матплотлиб', 'знак_сайкит',
       'пол', 'направление', 'уровень', 'есть_vk_id']
n = len(catvars)
# Пусть 1 - гипотеза принята, 0 - гипотеза отклонена (гипотеза независимости).
Matrix = []
tmp_l = []
for i in range(n):
    tmp_l = []
    for j in range(n):
        T12_4 = pd.crosstab(df[catvars[i]], df[catvars[j]])
        if st.chi2_contingency(T12_4)[1] > 0.05:
            tmp_l.append(1)
            #print('Гипотеза принята для ', catvars[i], ' и ', catvars[j])
        else:
            tmp_l.append(0)        
            #print('Гипотеза отклонена для ', catvars[i], ' и ', catvars[j])
    Matrix.append(tmp_l)
    
X =  dict(zip(catvars, Matrix))            
frame = pd.DataFrame(data = X, index = [catvars[i] for i in range(n)])
frame

,зн_переменная,зн_идентификатор,зн_функция,зн_класс,зн_объект,зн_оператор,зн_операция,зн_рекурсия,зн_массив,зн_выражение,...,знак_сайпай,знак_реквестс,знак_паймонго,знак_теворкс,знак_матплотлиб,знак_сайкит,пол,направление,уровень,есть_vk_id
зн_переменная,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,1,1,0,1,1
зн_идентификатор,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,1,1
зн_функция,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,1,1,0,1,1
зн_класс,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,0,0,1,1
зн_объект,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,0,0,1,1
зн_оператор,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,1,0,0,0,1
зн_операция,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,0,0,0,1
зн_рекурсия,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,1,0,0,1,1
зн_массив,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,0,0,1,1
зн_выражение,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,0,0,1,1


_Задание 6_. Для выявления связи между бинарной номинальной и количественной (ранговой) переменной испольуются критерии равенства средних (Стьюдента, Манна-Уитни) для независимых выборок. Если мы, к примеру, хотим понять, зависит ли средняя любовь к жареной картошке от пола, то нам необходимо сформировать две выборки: X - любовь к жареной картошке при условии, что пол женский и Y - любовь к жареной картошке при условии, что пол мужской. Затем проверить равенство средних в этих двух выборках (не забыв, предварительно, проверить гипотезу о нормальности). Величина эффекта (степень различия) в этом случае считается отдельно и называется дисперсионным анализом (ANOVA - analysis of variation). Это выходит за рамки данного задания.

Пример того, как выбрать значения одного столбца при условии на другой (допустим, "жареный\_картошка" при условии, что пол "Женский) приведён в файле "Python Day 3.ipynb".

Необходимо выбрать какую-либо количественную переменную и проверить, зависит ли её среднее от направления подготовки человека и от того, оставил ли он информацию о своей странице в ВК или нет. Проинтерпретировать результат.

In [26]:
one = df.loc[df['направление'] == 'IT', 'тушеный_кабачок']
two = df.loc[df['направление'] == 'Humanities', 'тушеный_кабачок']

x_o = st.normaltest(one).pvalue
x_t = st.normaltest(two).pvalue
print(x_o) # 0.22660332694299276
print(x_t) # 0.44016366518606276
# Являются нормальными
print(st.ttest_ind(one, two))
# statistic=-0.45965420692467623, pvalue=0.6504875880123757 > 0.05 нулевая гипотеза (матожидания равны) принята.
# Матожидания равны, среднее не зависит от направления подготовки.

0.22660332694299276
0.44016366518606276
Ttest_indResult(statistic=-0.45965420692467623, pvalue=0.6504875880123757)


In [28]:
one = df.loc[df['есть_vk_id'] == 1, 'тушеный_кабачок']
two = df.loc[df['есть_vk_id'] == 2, 'тушеный_кабачок']

x_o = st.normaltest(one).pvalue
x_t = st.normaltest(two).pvalue
print(x_o) # 0.2614013315935749
print(x_t) # 0.11801647918150276
# Являются нормальными
print(st.ttest_ind(one, two))
# statistic=0.45965420692467623, pvalue=0.6504875880123757 > 0.05 нулевая гипотеза (матожидания равны) принята.
# Матожидания равны, среднее не зависит от наличия vk id.

0.2614013315935749
0.11801647918150276
Ttest_indResult(statistic=0.45965420692467623, pvalue=0.6504875880123757)
